<a href="https://colab.research.google.com/github/ambicapvs/ambica_info5731_spring2021/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [1]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
!polyglot download sentiment2.en
!pip install twython

     |████████████████████████████████| 133kB 15.9MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52557 sha256=848dbd60561da77ff049b95f7b7a0138145d3bc93323eaff57b771fda748ba40
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot
     |████████████████████████████████| 235kB 16.9MB/s 
  Created wheel for pyicu: filename=PyICU-2.6-cp37-cp37m-linux_x86_64.whl size=1306434 sha256=e3c96a49e67c31372e95b997eb85106b60b2745d9d55dac2af9c38d04a91af6c
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
Successfully built pyicu
     |████████████████████████████████| 41.4MB 114kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834296 sha256=b613e2a5404a09307c25797647401da490024fbba3a0a5da533f9516e18b141f
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a

In [2]:
#loading data
import pandas as pd
dataframe = pd.read_csv("https://raw.githubusercontent.com/ambicapvs/ambica_info5731_spring2021/main/Q3(assig-3).csv")
dataframe

,ReviewID,Review,sentiment
0,1,"Every once in a while a movie comes, that trul...",positive
1,2,This is a movie that only those who have felt ...,positive
2,3,"Truly a masterpiece, The Best Hollywood film o...",positive
3,4,Joaquin Phoenix gives a tour de force performa...,positive
4,5,Most of the time movies are anticipated like t...,positive
...,...,...,...
101,102,"One of the worst movies I have ever seen. Ok, ...",negative
102,103,I didnt really know much about this movie. I h...,negative
103,104,Film was directed and produced poorly. Opportu...,negative
104,105,"Wow, what a movie! I have to admit, When I fir...",positive


In [5]:
#sentiment term extraction and frequency
import nltk
import requests
from textblob import TextBlob
data = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
a = data.text
stop = a.split()
#stopwords
dataframe['stopwords'] = dataframe["Review"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#lowercase
dataframe["lower_case"] = dataframe['stopwords'].apply(lambda x: " ".join(x.lower() for x in x.split()))
terms = []
from polyglot.text import Text
for items in dataframe["lower_case"]:
  item = Text(items)
  for word in item.words:
    if word.polarity != 0:
      terms.append(word)
      #print(word, word.polarity)

from collections import Counter
frequencies = Counter(terms)
rankbyfreq = list(frequencies.most_common())
#print(rankbyfreq)
dataframerank = pd.DataFrame(rankbyfreq, columns =['term', 'frequency'])
dataframerank

,term,frequency
0,joker,143
1,like,54
2,good,54
3,best,30
4,great,28
...,...,...
518,disappointing,1
519,creepy,1
520,unrealistic,1
521,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [11]:
#Sentiment Analysis with TextBlob
from textblob import TextBlob
textblobsentiment = []
for item in dataframe['lower_case']:
  polarity = TextBlob(item).sentiment.polarity
  if polarity > 0 :
    textblobsentiment.append("positive")
  elif polarity < 0:
    textblobsentiment.append("negative")
  elif polarity == 0:
    textblobsentiment.append("neutral")
sentimentDataframe = pd.DataFrame(list(zip(dataframe['lower_case'], dataframe['sentiment'], textblobsentiment)), columns = ['Review', 'Actual', 'TextBlobPredicted'])
sentimentDataframe

,Review,Actual,TextBlobPredicted
0,"every movie comes, truly makes impact. joaquin...",positive,negative
1,this movie felt alone isolated truly relate it...,positive,neutral
2,"truly masterpiece, the best hollywood film 201...",positive,positive
3,joaquin phoenix gives tour de force performanc...,positive,negative
4,most time movies anticipated like end falling ...,positive,positive
...,...,...,...
101,"one worst movies i ever seen. ok, maybe hype s...",negative,positive
102,i didnt really know much movie. i hadnt seen t...,negative,positive
103,film directed produced poorly. opportunity tak...,negative,negative
104,"wow, movie! i admit, when i first heard joaqui...",positive,positive


In [12]:
#Calculate Accuracy and F1 Score for TextBlob Sentiment Analysis
from sklearn.metrics import f1_score, accuracy_score
txtblobaccuracy = accuracy_score(sentimentDataframe['Actual'], sentimentDataframe['TextBlobPredicted'])*100
txtblobscore = f1_score(sentimentDataframe['Actual'], sentimentDataframe['TextBlobPredicted'], average = 'macro')
print("TextBlob \nAccuracy: ",txtblobaccuracy,"\nF1 Score: ", txtblobscore)

TextBlob 
Accuracy:  59.43396226415094 
F1 Score:  0.4642388150862728


In [13]:
#VADER Sentiment Analysis
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vadersentiment = []
for line in dataframe['lower_case']:
  polarity = vader.polarity_scores(line)
  if polarity['compound'] > 0 :
    vadersentiment.append("positive")
  elif polarity['compound'] < 0:
    vadersentiment.append("negative")
  elif polarity['compound'] == 0:
    vadersentiment.append("neutral")
#vadersentiment
sentimentDataframe['VaderPredicted'] = vadersentiment
sentimentDataframe

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Review,Actual,TextBlobPredicted,VaderPredicted
0,"every movie comes, truly makes impact. joaquin...",positive,negative,positive
1,this movie felt alone isolated truly relate it...,positive,neutral,positive
2,"truly masterpiece, the best hollywood film 201...",positive,positive,positive
3,joaquin phoenix gives tour de force performanc...,positive,negative,positive
4,most time movies anticipated like end falling ...,positive,positive,negative
...,...,...,...,...
101,"one worst movies i ever seen. ok, maybe hype s...",negative,positive,negative
102,i didnt really know much movie. i hadnt seen t...,negative,positive,positive
103,film directed produced poorly. opportunity tak...,negative,negative,negative
104,"wow, movie! i admit, when i first heard joaqui...",positive,positive,positive


In [14]:
#Calculate Accuracy and F1 Score for VADER Sentiment Analysis
vaderaccuracy = accuracy_score(sentimentDataframe['Actual'], sentimentDataframe['VaderPredicted'])*100
vaderscore = f1_score(sentimentDataframe['Actual'], sentimentDataframe['VaderPredicted'], average = 'macro')
print("VADER \nAccuracy: ",vaderaccuracy,"\nF1 Score: ", vaderscore)

VADER 
Accuracy:  63.20754716981132 
F1 Score:  0.4611599551358587


In [16]:
#SVM Analysis

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

#Reference - https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34
Train_X, Test_X, Train_Y, Test_Y = train_test_split(dataframe['Review'],dataframe['sentiment'],test_size=0.3)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Tfidf_vect = TfidfVectorizer(max_features= 5000, use_idf = True )
Train_X_vectors = Tfidf_vect.fit_transform(Train_X)
Test_X_vectors = Tfidf_vect.transform(Test_X)


from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='linear')
svm_model.fit(Train_X_vectors, Train_Y)
predicted = svm_model.predict(Test_X_vectors)
svmreport = classification_report(Test_Y, predicted, output_dict=True, zero_division=0  )
print("SVM \nAccuracy: ",accuracy_score(predicted, Test_Y)*100, "\nF1 score", svmreport['macro avg']['f1-score'], "\n\n" )
svmreport


SVM 
Accuracy:  68.75 
F1 score 0.5186104218362283 




{'0': {'f1-score': 0.7096774193548386,
  'precision': 0.6470588235294118,
  'recall': 0.7857142857142857,
  'support': 14},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 7},
 '2': {'f1-score': 0.846153846153846,
  'precision': 0.7333333333333333,
  'recall': 1.0,
  'support': 11},
 'accuracy': 0.6875,
 'macro avg': {'f1-score': 0.5186104218362283,
  'precision': 0.46013071895424834,
  'recall': 0.5952380952380952,
  'support': 32},
 'weighted avg': {'f1-score': 0.6013492555831265,
  'precision': 0.535171568627451,
  'recall': 0.6875,
  'support': 32}}

**Performance Comparison**

The Accuracy of SVM is higher than the scores of VADER and textblob. 
So, if accuracy is used as a measure SVM is preferred over VADER and Textblob in the order. 

The F1 score of SVM is higher than the scores of textblob and VADER. 
So, if F1 score is used as a measure SVM is preferred over Textblob and VADER in the order. 


ref:  https://medium.com/analytics-vidhya/accuracy-vs-f1-score-6258237beca2 


---



